<a href="https://colab.research.google.com/github/MissoumYoucef/Kaggle_Competition/blob/main/Facial_Keypoints_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'facial-keypoints-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3486%2F31310%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240404%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240404T000520Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5906d777f8d549ab60e105307d420140ba8a9b9d4713ad3fe790d5850f15a7a392e991d05b79a0feb1e3aef9233900a874d1abf713c8e29bb8b23fcf9ab11e172d1614714e38614a34bcd33bb10a99c33e9c803c72fdd7bfe871fa70a2a622566b188bea9236a165b68a3b94222a93090a78787582e84e1e898b35389f3b040fb406c0addbbdac8a81e0fe00a875af367077d56b5aef522e57d0563444edbfa072c7b6a887bd63a73411955b6eaea959c3f33dca2ccc5107fb45180797c0278e339783dde8d7a73c82fa1c5bac21a3c1cd41b828b23356b128d156f26299844373222487929942076774d1a7a31ce829cbf2063b2129dd97ff04f55199f7ac43'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
from keras.layers import Conv2D,Dropout,Dense,Flatten
from keras.models import Sequential
from keras.layers import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

2024-04-03 23:28:00.857140: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 23:28:00.857291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 23:28:01.055037: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
train_data = pd.read_csv('/kaggle/input/facial-keypoints-detection/training.zip')
test_data = pd.read_csv('/kaggle/input/facial-keypoints-detection/test.zip')

In [ ]:
train_data.shape,test_data.shape

((7049, 31), (1783, 2))

In [ ]:
train_data.columns

Index(['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x',
       'right_eye_center_y', 'left_eye_inner_corner_x',
       'left_eye_inner_corner_y', 'left_eye_outer_corner_x',
       'left_eye_outer_corner_y', 'right_eye_inner_corner_x',
       'right_eye_inner_corner_y', 'right_eye_outer_corner_x',
       'right_eye_outer_corner_y', 'left_eyebrow_inner_end_x',
       'left_eyebrow_inner_end_y', 'left_eyebrow_outer_end_x',
       'left_eyebrow_outer_end_y', 'right_eyebrow_inner_end_x',
       'right_eyebrow_inner_end_y', 'right_eyebrow_outer_end_x',
       'right_eyebrow_outer_end_y', 'nose_tip_x', 'nose_tip_y',
       'mouth_left_corner_x', 'mouth_left_corner_y', 'mouth_right_corner_x',
       'mouth_right_corner_y', 'mouth_center_top_lip_x',
       'mouth_center_top_lip_y', 'mouth_center_bottom_lip_x',
       'mouth_center_bottom_lip_y', 'Image'],
      dtype='object')

In [ ]:
# Extract the image data and keypoint coordinates
images = train_data['Image'].apply(lambda x: np.array(x.split(' '), dtype=int))

In [ ]:
null_counts=train_data.isna().sum()
# null_counts

In [ ]:
non_much_Null_values=null_counts.index
non_much_Null_values = [col for col in non_much_Null_values if col != 'Image']
# non_much_Null_values

In [ ]:
for feature in non_much_Null_values:
    median_value = train_data[feature].median()
    train_data[feature].fillna(median_value, inplace=True)

/tmp/ipykernel_33/1613402553.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data[feature].fillna(median_value, inplace=True)
/tmp/ipykernel_33/1613402553.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [ ]:
import albumentations as A

# Define the augmentation transforms
transforms = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.Transpose(),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(blur_limit=(3, 7), p=0.5),
    A.Normalize()
])

In [ ]:
# Create a new DataFrame to store the augmented data
augmented_data = pd.DataFrame(columns=train_data.columns)

In [ ]:
image_str=train_data['Image'][0]

In [ ]:
x=np.array([int(x) for x in image_str.split()])
x

array([238, 236, 237, ...,  70,  75,  90])

In [ ]:
x=x.reshape(96,96,1)
x

array([[[238],
        [236],
        [237],
        ...,
        [250],
        [250],
        [250]],

       [[235],
        [238],
        [236],
        ...,
        [249],
        [250],
        [251]],

       [[237],
        [236],
        [237],
        ...,
        [251],
        [251],
        [250]],

       ...,

       [[186],
        [183],
        [181],
        ...,
        [ 52],
        [ 57],
        [ 60]],

       [[189],
        [188],
        [207],
        ...,
        [ 61],
        [ 69],
        [ 78]],

       [[191],
        [184],
        [184],
        ...,
        [ 70],
        [ 75],
        [ 90]]])

In [ ]:
# Extract the image and keypoint data
for index, row in train_data.iterrows():
    image_str = row['Image']
    image = np.array([int(x) for x in image_str.split()])
    image = image.reshape(96, 96, 1)
    keypoints = np.array([
        (row['left_eye_center_x'], row['left_eye_center_y'], 0, 1),
        (row['right_eye_center_x'], row['right_eye_center_y'], 0, 1),
        (row['left_eye_inner_corner_x'], row['left_eye_inner_corner_y'], 0, 1),
        (row['left_eye_outer_corner_x'], row['left_eye_outer_corner_y'], 0, 1),
        (row['right_eye_inner_corner_x'], row['right_eye_inner_corner_y'], 0, 1),
        (row['right_eye_outer_corner_x'], row['right_eye_outer_corner_y'], 0, 1),
        (row['left_eyebrow_inner_end_x'], row['left_eyebrow_inner_end_y'], 0, 1),
        (row['left_eyebrow_outer_end_x'], row['left_eyebrow_outer_end_y'], 0, 1),
        (row['right_eyebrow_inner_end_x'], row['right_eyebrow_inner_end_y'], 0, 1),
        (row['right_eyebrow_outer_end_x'], row['right_eyebrow_outer_end_y'], 0, 1),
        (row['nose_tip_x'], row['nose_tip_y'], 0, 1),
        (row['mouth_left_corner_x'], row['mouth_left_corner_y'], 0, 1),
        (row['mouth_right_corner_x'], row['mouth_right_corner_y'], 0, 1),
        (row['mouth_center_top_lip_x'], row['mouth_center_top_lip_y'], 0, 1),
        (row['mouth_center_bottom_lip_x'], row['mouth_center_bottom_lip_y'], 0, 1)
    ])

    # Apply the augmentation transforms
    augmented = transforms(image=image, keypoints=keypoints)
    augmented_image = augmented['image']
    augmented_keypoints = augmented['keypoints']

    # Create a new row in the augmented_data DataFrame
    new_row = {
        'left_eye_center_x': augmented_keypoints[0][0],
        'left_eye_center_y': augmented_keypoints[0][1],
        'right_eye_center_x': augmented_keypoints[1][0],
        'right_eye_center_y': augmented_keypoints[1][1],
        'left_eye_inner_corner_x': augmented_keypoints[2][0],
        'left_eye_inner_corner_y': augmented_keypoints[2][1],
        'left_eye_outer_corner_x': augmented_keypoints[3][0],
        'left_eye_outer_corner_y': augmented_keypoints[3][1],
        'right_eye_inner_corner_x': augmented_keypoints[4][0],
        'right_eye_inner_corner_y': augmented_keypoints[4][1],
        'right_eye_outer_corner_x': augmented_keypoints[5][0],
        'right_eye_outer_corner_y': augmented_keypoints[5][1],
        'left_eyebrow_inner_end_x': augmented_keypoints[6][0],
        'left_eyebrow_inner_end_y': augmented_keypoints[6][1],
        'left_eyebrow_outer_end_x': augmented_keypoints[7][0],
        'left_eyebrow_outer_end_y': augmented_keypoints[7][1],
        'right_eyebrow_inner_end_x': augmented_keypoints[8][0],
        'right_eyebrow_inner_end_y': augmented_keypoints[8][1],
        'right_eyebrow_outer_end_x': augmented_keypoints[9][0],
        'right_eyebrow_outer_end_y': augmented_keypoints[9][1],
        'nose_tip_x': augmented_keypoints[10][0],
        'nose_tip_y': augmented_keypoints[10][1],
        'mouth_left_corner_x': augmented_keypoints[11][0],
        'mouth_left_corner_y': augmented_keypoints[11][1],
        'mouth_right_corner_x': augmented_keypoints[12][0],
        'mouth_right_corner_y': augmented_keypoints[12][1],
        'mouth_center_top_lip_x': augmented_keypoints[13][0],
        'mouth_center_top_lip_y': augmented_keypoints[13][1],
        'mouth_center_bottom_lip_x': augmented_keypoints[14][0],
        'mouth_center_bottom_lip_y': augmented_keypoints[14][1],
        'Image': augmented_image
    }


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/imgwarp.cpp:1820: error: (-215:Assertion failed) ifunc != 0 in function 'remap'


In [ ]:
# Apply the augmentation transforms to the data
for index, row in train_data.iterrows():
    # Extract the image and keypoint data
    image_str = row['Image']
    image = np.array([int(x) for x in image_str.split()])
    image = image.reshape(96, 96, 1)
    keypoints = np.array([
    row['left_eye_center_x'], row['left_eye_center_y'],
    row['right_eye_center_x'], row['right_eye_center_y'],
    row['left_eye_inner_corner_x'], row['left_eye_inner_corner_y'],
    row['left_eye_outer_corner_x'], row['left_eye_outer_corner_y'],
    row['right_eye_inner_corner_x'], row['right_eye_inner_corner_y'],
    row['right_eye_outer_corner_x'], row['right_eye_outer_corner_y'],
    row['left_eyebrow_inner_end_x'], row['left_eyebrow_inner_end_y'],
    row['left_eyebrow_outer_end_x'], row['left_eyebrow_outer_end_y'],
    row['right_eyebrow_inner_end_x'], row['right_eyebrow_inner_end_y'],
    row['right_eyebrow_outer_end_x'], row['right_eyebrow_outer_end_y'],
    row['nose_tip_x'], row['nose_tip_y'],
    row['mouth_left_corner_x'], row['mouth_left_corner_y'],
    row['mouth_right_corner_x'], row['mouth_right_corner_y'],
    row['mouth_center_top_lip_x'], row['mouth_center_top_lip_y'],
    row['mouth_center_bottom_lip_x'], row['mouth_center_bottom_lip_y']
    ]).reshape(-1, 2)
    # Apply the augmentation transforms
    augmented = transforms(image=image, keypoints=keypoints)
    augmented_image = augmented['image']
    augmented_keypoints = augmented['keypoints']

    # Create a new row in the augmented_data DataFrame
    new_row = {
    'left_eye_center_x': augmented_keypoints[0],
    'left_eye_center_y': augmented_keypoints[1],
    'right_eye_center_x': augmented_keypoints[2],
    'right_eye_center_y': augmented_keypoints[3],
    'left_eye_inner_corner_x': augmented_keypoints[4],
    'left_eye_inner_corner_y': augmented_keypoints[5],
    'left_eye_outer_corner_x': augmented_keypoints[6],
    'left_eye_outer_corner_y': augmented_keypoints[7],
    'right_eye_inner_corner_x': augmented_keypoints[8],
    'right_eye_inner_corner_y': augmented_keypoints[9],
    'right_eye_outer_corner_x': augmented_keypoints[10],
    'right_eye_outer_corner_y': augmented_keypoints[11],
    'left_eyebrow_inner_end_x': augmented_keypoints[12],
    'left_eyebrow_inner_end_y': augmented_keypoints[13],
    'left_eyebrow_outer_end_x': augmented_keypoints[14],
    'left_eyebrow_outer_end_y': augmented_keypoints[15],
    'right_eyebrow_inner_end_x': augmented_keypoints[16],
    'right_eyebrow_inner_end_y': augmented_keypoints[17],
    'right_eyebrow_outer_end_x': augmented_keypoints[18],
    'right_eyebrow_outer_end_y': augmented_keypoints[19],
    'nose_tip_x': augmented_keypoints[20],
    'nose_tip_y': augmented_keypoints[21],
    'mouth_left_corner_x': augmented_keypoints[22],
    'mouth_left_corner_y': augmented_keypoints[23],
    'mouth_right_corner_x': augmented_keypoints[24],
    'mouth_right_corner_y': augmented_keypoints[25],
    'mouth_center_top_lip_x': augmented_keypoints[26],
    'mouth_center_top_lip_y': augmented_keypoints[27],
    'mouth_center_bottom_lip_x': augmented_keypoints[28],
    'mouth_center_bottom_lip_y': augmented_keypoints[29],
    'Image': augmented_image
    }

    augmented_data = augmented_data.append(new_row, ignore_index=True)

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:


# Save the augmented data to a new file
augmented_data.to_csv('augmented_train_data.csv', index=False)


In [ ]:
keypoints = train_data.drop('Image', axis=1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, keypoints, test_size=0.2, random_state=42)

In [ ]:
# Reshape the image data
X_train = np.array([x.reshape(96, 96, 1) for x in X_train])
X_val = np.array([x.reshape(96, 96, 1) for x in X_val])

In [ ]:
X_train.shape

(5639, 96, 96, 1)

In [ ]:
import tensorflow as tf
tf_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
# Assuming X_train is your grayscale image dataset with shape (num_images, 96, 96, 1)
X_train_rgb = tf.image.grayscale_to_rgb(tf_tensor)

In [ ]:
X_train_rgb.shape

TensorShape([5639, 96, 96, 3])

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2S,ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
from tensorflow.keras.applications import VGG19

vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
for layer in vgg19_model.layers:
    layer.trainable = False

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
from tensorflow.keras.applications import DenseNet121

densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
for layer in densenet_model.layers:
    layer.trainable = False

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
from tensorflow.keras.applications import MobileNet
mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
for layer in mobilenet_model.layers:
    layer.trainable = False

/tmp/ipykernel_33/1187213090.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(96, 96, 3))


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB4
efficientnet_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
for layer in efficientnet_model.layers:
    layer.trainable = False

71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
model = ResNet50(include_top=True, weights='imagenet', input_shape=(224, 224, 3))
resnext_model = ResNet50(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
for layer in resnext_model.layers:
    layer.trainable = False

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
def trainfun(model):
    model = models.Sequential([
        model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(30)  # Assuming a binary classification problem
    ])
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])
    return model

In [ ]:
model=trainfun(resnext_model)
model.fit(X_train_rgb,y_train,epochs =5,batch_size = 256,validation_split = 0.2)

Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 102s 5s/step - loss: 1484.4110 - mae: 32.7555 - val_loss: 193.4441 - val_mae: 9.5057
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - loss: 148.9010 - mae: 9.3745 - val_loss: 67.4477 - val_mae: 6.2839
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - loss: 77.5290 - mae: 6.7403 - val_loss: 37.6809 - val_mae: 4.5498
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - loss: 51.8672 - mae: 5.4728 - val_loss: 28.2894 - val_mae: 3.8733
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - loss: 43.1938 - mae: 4.9533 - val_loss: 23.2202 - val_mae: 3.4903


In [ ]:
model2=trainfun(efficientnet_model)
model2.fit(X_train_rgb,y_train,epochs =5,batch_size = 256,validation_split = 0.2)

Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 145s 7s/step - loss: 2150.8955 - mae: 42.2947 - val_loss: 395.5820 - val_mae: 15.5772
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 137s 6s/step - loss: 280.0997 - mae: 13.1886 - val_loss: 113.1493 - val_mae: 8.2016
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - loss: 98.6514 - mae: 7.5761 - val_loss: 50.6105 - val_mae: 5.0952
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 141s 6s/step - loss: 55.5512 - mae: 5.3673 - val_loss: 38.5260 - val_mae: 4.2299
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 143s 7s/step - loss: 43.2571 - mae: 4.6454 - val_loss: 32.3522 - val_mae: 3.9405


In [ ]:
model3=trainfun(vgg19_model)
model3.fit(X_train_rgb,y_train,epochs =5,batch_size = 256,validation_split = 0.2)

Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 361s 20s/step - loss: 1466.9417 - mae: 32.2074 - val_loss: 273.3984 - val_mae: 11.9073
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 384s 20s/step - loss: 229.0069 - mae: 11.6447 - val_loss: 138.8029 - val_mae: 9.0945
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 360s 20s/step - loss: 150.7518 - mae: 9.4315 - val_loss: 108.7550 - val_mae: 8.0666
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 382s 20s/step - loss: 123.6641 - mae: 8.5643 - val_loss: 91.6880 - val_mae: 7.3585
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 382s 20s/step - loss: 105.6342 - mae: 7.8627 - val_loss: 80.7559 - val_mae: 6.8853


In [ ]:
model4=trainfun(densenet_model)
model4.fit(X_train_rgb,y_train,epochs =5,batch_size = 256,validation_split = 0.2)

Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 117s 6s/step - loss: 1120.9617 - mae: 26.4212 - val_loss: 231.8927 - val_mae: 12.2796
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 96s 5s/step - loss: 228.8987 - mae: 11.9610 - val_loss: 124.3937 - val_mae: 8.5608
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - loss: 141.6135 - mae: 9.2825 - val_loss: 80.5321 - val_mae: 6.7022
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 143s 5s/step - loss: 98.9215 - mae: 7.6882 - val_loss: 56.5455 - val_mae: 5.6161
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - loss: 73.7886 - mae: 6.6371 - val_loss: 45.3091 - val_mae: 5.0742


In [ ]:
model5=trainfun(mobilenet_model)
model5.fit(X_train_rgb,y_train,epochs =5,batch_size = 256,validation_split = 0.2)

Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - loss: 1892.3571 - mae: 38.6905 - val_loss: 209.7957 - val_mae: 11.5428
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - loss: 224.1553 - mae: 11.2316 - val_loss: 83.1424 - val_mae: 7.6923
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - loss: 70.9693 - mae: 6.6856 - val_loss: 30.1498 - val_mae: 4.1475
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - loss: 37.3103 - mae: 4.6533 - val_loss: 23.2466 - val_mae: 3.5529
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - loss: 31.7413 - mae: 4.3009 - val_loss: 19.3552 - val_mae: 3.1858


In [ ]:
#preparing test data
timag = []
for i in range(0,1783):
    timg = test_data['Image'][i].split(' ')
    timg = ['0' if x == '' else x for x in timg]

    timag.append(timg)

In [ ]:
timage_list = np.array(timag,dtype = 'float')
X_test = timage_list.reshape(-1,96,96,1)

In [ ]:
X_test.shape

(1783, 96, 96, 1)

In [ ]:
tf_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
# Assuming X_train is your grayscale image dataset with shape (num_images, 96, 96, 1)
X_test = tf.image.grayscale_to_rgb(tf_tensor)

In [ ]:
pred1 = model.predict(X_test)
pred2 = model2.predict(X_test)
pred3 = model3.predict(X_test)
pred4 = model4.predict(X_test)
pred5 = model5.predict(X_test)

56/56 ━━━━━━━━━━━━━━━━━━━━ 28s 500ms/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 32s 564ms/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 34s 530ms/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 132ms/step


In [ ]:
pred=(pred5+pred4)/2

In [ ]:
lookid_data = pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')
lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(pred)
rowid = lookid_data['RowId']
rowid=list(rowid)
feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])
rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(preded,name = 'Location')
submission = pd.concat([rowid,loc],axis = 1)
submission.to_csv('/kaggle/working/face_key_detection_submission.csv',index = False)

In [ ]:
fd=pd.read_csv('/kaggle/working/face_key_detection_submission.csv')

In [ ]:
fd[fd['Location'] > 96]

,RowId,Location
9462,9463,96.039640
9477,9478,97.489160
15121,15122,96.277070
22483,22484,103.636505
23795,23796,99.541565


In [ ]:
fd.loc[fd['Location'] > 96, 'Location'] = 95
submission = fd

In [ ]:
submission.to_csv('/kaggle/working/face_key_detection_submission.csv',index = False)